In [1]:
import pandas as pd
import gensim
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
import csv
from datetime import datetime, timedelta
import html.parser
html_parser = html.parser.HTMLParser()
from ftfy import fix_text # Fix any unicode problem
import re
import itertools
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
from ekphrasis.classes.segmenter import Segmenter
from ekphrasis.classes.spellcorrect import SpellCorrector
from ekphrasis.classes.tokenizer import SocialTokenizer
from autocorrect import spell
sp = SpellCorrector(corpus="english")

[nltk_data] Downloading package stopwords to /Users/faten/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Reading english - 1grams ...


In [2]:
# After scandal
df = pd.read_csv("output_month_after_scandal.csv",  sep=";",engine='python', header=None, skip_blank_lines=1) # Read the folder of tweets
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header
df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
df['Date'] = [d.date() for d in df['date']]
df['Time'] = [d.time() for d in df['date']]
#df = df.drop(df.columns[[1,4,8,9,10,11,12,13,14]], axis=1) #drop invalid columns ('geo','nan','date')
tweets = df['text'].tolist()

In [3]:
tweets = tweets[0:20]
tweets

['Volkswagen Ordered To Recall 500K Vehicles Over Its Own Malicious Programming http:// bit.ly/1OjPG0x',
 'What. The. Actual. Fuck. http:// nyti.ms/1iouQjb #volkswagen #emissions (h/t @alexandernl )',
 '@latimesphotos @Volkswagen Have we just given up on English?',
 'Seriously though, what the fuck Volkswagen',
 'Volkswagen , Audi accused of using software to cheat US diesel emissions tests http:// fb.me/3iL9CIqZy',
 'Jorge R Ruiz: Volkswagen , Audi accused of using software to cheat US diesel emissions tests: In a... http:// yhoo.it/1QN36Ap @Tech_Minute',
 '#vwbeetle Volkswagen : Beetle - Classic 1969 volkswagon convertible: $5,000.00 (1 Bid)End Date: Friday Sep-25... http:// ebay.to/1gyU0uf',
 '#BLG VW cheated on emissions tests: The EPA says Volkswagen installed software to cheat on emissions tests. US... http:// usat.ly/1Pa00p4',
 "VW cheated on emissions tests: The EPA says Volkswagen installed software to cheat on emissions tests. USA TODAY's Chris Woodyard rep...",
 'VW cheated 

In [4]:
#Escaping HTML characters + Fixing any unicode problem
# for example : "&lt;3" is converted to '<3'
parser_tweets = []
for tweet in tweets:
    tweet = html_parser.unescape(tweet)
    tweet = fix_text(tweet) # Fix any unicode problem
    parser_tweets.append(tweet)
parser_tweets

/Users/faten/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.
  """


['Volkswagen Ordered To Recall 500K Vehicles Over Its Own Malicious Programming http:// bit.ly/1OjPG0x',
 'What. The. Actual. Fuck. http:// nyti.ms/1iouQjb #volkswagen #emissions (h/t @alexandernl )',
 '@latimesphotos @Volkswagen Have we just given up on English?',
 'Seriously though, what the fuck Volkswagen',
 'Volkswagen , Audi accused of using software to cheat US diesel emissions tests http:// fb.me/3iL9CIqZy',
 'Jorge R Ruiz: Volkswagen , Audi accused of using software to cheat US diesel emissions tests: In a... http:// yhoo.it/1QN36Ap @Tech_Minute',
 '#vwbeetle Volkswagen : Beetle - Classic 1969 volkswagon convertible: $5,000.00 (1 Bid)End Date: Friday Sep-25... http:// ebay.to/1gyU0uf',
 '#BLG VW cheated on emissions tests: The EPA says Volkswagen installed software to cheat on emissions tests. US... http:// usat.ly/1Pa00p4',
 "VW cheated on emissions tests: The EPA says Volkswagen installed software to cheat on emissions tests. USA TODAY's Chris Woodyard rep...",
 'VW cheated 

In [5]:
#Removal of URLs
no_url_tweets=[]
for tweet in parser_tweets : 
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet, flags=re.MULTILINE)
    no_url_tweets.append(tweet)
no_url_tweets

['Volkswagen Ordered To Recall 500K Vehicles Over Its Own Malicious Programming ',
 'What. The. Actual. Fuck. ',
 '@latimesphotos @Volkswagen Have we just given up on English?',
 'Seriously though, what the fuck Volkswagen',
 'Volkswagen , Audi accused of using software to cheat US diesel emissions tests ',
 'Jorge R Ruiz: Volkswagen , Audi accused of using software to cheat US diesel emissions tests: In a... ',
 '#vwbeetle Volkswagen : Beetle - Classic 1969 volkswagon convertible: $5,000.00 (1 Bid)End Date: Friday Sep-25... ',
 '#BLG VW cheated on emissions tests: The EPA says Volkswagen installed software to cheat on emissions tests. US... ',
 "VW cheated on emissions tests: The EPA says Volkswagen installed software to cheat on emissions tests. USA TODAY's Chris Woodyard rep...",
 'VW cheated on emissions tests: The EPA says Volkswagen installed software to cheat on emissions tests. USA TOD... ',
 "EPA: Volkswagen used 'defeat device' to illegally skirt air-pollution controls - Wash

In [6]:
#define the dictionary of APPOSTOPHES
APPOSTOPHES = {"aren't" : "are not","can't" : "cannot","couldn't" : "could not","didn't" : "did not","doesn't" : "does not","don't" : "do not",
"hadn't" : "had not","hasn't" : "has not","haven't" : "have not","he'd" : "he would","he'll" : "he will","he's" : "he is","i'd" : "I would",
"i'd" : "I had","i'll" : "I will","i'm" : "I am","isn't" : "is not","it's" : "it is","it'll":"it will","i've" : "I have","let's" : "let us","mightn't" : "might not","mustn't" : "must not","shan't" : "shall not","she'd" : "she would","she'll" : "she will","she's" : "she is","shouldn't" : "should not","that's" : "that is","there's" : "there is","they'd" : "they would","they'll" : "they will","they're" : "they are","they've" : "they have","we'd" : "we would","we're" : "we are","weren't" : "were not","we've" : "we have","what'll" : "what will","what're" : "what are","what's" : "what is","what've" : "what have","where's" : "where is","who'd" : "who would","who'll" : "who will","who're" : "who are","who's" : "who is","who've" : "who have","won't" : "will not","wouldn't" : "would not","you'd" : "you would","you'll" : "you will","you're" : "you are","you've" : "you have","'re": " are","wasn't": "was not","we'll":" will",
"aren’t" : "are not","can’t" : "cannot","couldn’t" : "could not","didn’t" : "did not","doesn’t" : "does not","don’t" : "do not",
"hadn’t" : "had not","hasn’t" : "has not","haven’t" : "have not","he’d" : "he would","he’ll" : "he will","he’s" : "he is","i’d" : "I would",
"i’d" : "I had","i’ll" : "I will","i’m" : "I am","isn’t" : "is not","it’s" : "it is","it’ll":"it will","i’ve" : "I have","let’s" : "let us","mightn’t" : "might not","mustn’t" : "must not","shan’t" : "shall not","she’d" : "she would","she’ll" : "she will","she’s" : "she is","shouldn’t" : "should not","that’s" : "that is","there’s" : "there is","they’d" : "they would","they’ll" : "they will","they’re" : "they are","they’ve" : "they have","we’d" : "we would","we’re" : "we are","weren’t" : "were not","we’ve" : "we have","what’ll" : "what will","what’re" : "what are","what’s" : "what is","what’ve" : "what have","where’s" : "where is","who’d" : "who would","who’ll" : "who will","who’re" : "who are","who’s" : "who is","who’ve" : "who have","won’t" : "will not","wouldn’t" : "would not","you’d" : "you would","you’ll" : "you will","you’re" : "you are","you’ve" : "you have","’re": " are","wasn’t": "was not","we’ll":" will","didn’t": "did not"
}

In [7]:
no_appos_tweets = []
for tweet in no_url_tweets :
    words = tweet.split()
    tweet = [APPOSTOPHES[word] if word in APPOSTOPHES else word for word in words]
    tweet = " ".join(tweet)
    no_appos_tweets.append(tweet)
no_appos_tweets

['Volkswagen Ordered To Recall 500K Vehicles Over Its Own Malicious Programming',
 'What. The. Actual. Fuck.',
 '@latimesphotos @Volkswagen Have we just given up on English?',
 'Seriously though, what the fuck Volkswagen',
 'Volkswagen , Audi accused of using software to cheat US diesel emissions tests',
 'Jorge R Ruiz: Volkswagen , Audi accused of using software to cheat US diesel emissions tests: In a...',
 '#vwbeetle Volkswagen : Beetle - Classic 1969 volkswagon convertible: $5,000.00 (1 Bid)End Date: Friday Sep-25...',
 '#BLG VW cheated on emissions tests: The EPA says Volkswagen installed software to cheat on emissions tests. US...',
 "VW cheated on emissions tests: The EPA says Volkswagen installed software to cheat on emissions tests. USA TODAY's Chris Woodyard rep...",
 'VW cheated on emissions tests: The EPA says Volkswagen installed software to cheat on emissions tests. USA TOD...',
 "EPA: Volkswagen used 'defeat device' to illegally skirt air-pollution controls - Washington 

In [8]:
#Removal of Punctuations
def remove_ponctuation(tweet):
    # define punctuation
    punctuations = '''!()-![]{};:+'"\,<>./?@#$%^&*_~'''
    # remove punctuation from the tweet
    no_punct = ""
    for word in tweet:
        if word not in punctuations:
            no_punct = no_punct + word
    return no_punct   

no_punct_tweets = []
for tweet in no_appos_tweets:
    tweet = remove_ponctuation(tweet)
    no_punct_tweets.append(tweet)
no_punct_tweets

['Volkswagen Ordered To Recall 500K Vehicles Over Its Own Malicious Programming',
 'What The Actual Fuck',
 'latimesphotos Volkswagen Have we just given up on English',
 'Seriously though what the fuck Volkswagen',
 'Volkswagen  Audi accused of using software to cheat US diesel emissions tests',
 'Jorge R Ruiz Volkswagen  Audi accused of using software to cheat US diesel emissions tests In a',
 'vwbeetle Volkswagen  Beetle  Classic 1969 volkswagon convertible 500000 1 BidEnd Date Friday Sep25',
 'BLG VW cheated on emissions tests The EPA says Volkswagen installed software to cheat on emissions tests US',
 'VW cheated on emissions tests The EPA says Volkswagen installed software to cheat on emissions tests USA TODAYs Chris Woodyard rep',
 'VW cheated on emissions tests The EPA says Volkswagen installed software to cheat on emissions tests USA TOD',
 'EPA Volkswagen used defeat device to illegally skirt airpollution controls  Washington Post',
 'VW cheated on emissions tests The EPA says

In [9]:
#Removal of digits
no_digit_tweets = []
for tweet in no_punct_tweets : 
    tweet = ''.join([i for i in tweet if not i.isdigit()])
    no_digit_tweets.append(tweet)
no_digit_tweets

['Volkswagen Ordered To Recall K Vehicles Over Its Own Malicious Programming',
 'What The Actual Fuck',
 'latimesphotos Volkswagen Have we just given up on English',
 'Seriously though what the fuck Volkswagen',
 'Volkswagen  Audi accused of using software to cheat US diesel emissions tests',
 'Jorge R Ruiz Volkswagen  Audi accused of using software to cheat US diesel emissions tests In a',
 'vwbeetle Volkswagen  Beetle  Classic  volkswagon convertible   BidEnd Date Friday Sep',
 'BLG VW cheated on emissions tests The EPA says Volkswagen installed software to cheat on emissions tests US',
 'VW cheated on emissions tests The EPA says Volkswagen installed software to cheat on emissions tests USA TODAYs Chris Woodyard rep',
 'VW cheated on emissions tests The EPA says Volkswagen installed software to cheat on emissions tests USA TOD',
 'EPA Volkswagen used defeat device to illegally skirt airpollution controls  Washington Post',
 'VW cheated on emissions tests The EPA says Volkswagen inst

In [10]:
#Removal of  emoticons, symbols & pictographs, transport & map symbols and flags (iOS)
no_emoji_tweets = []
for tweet in no_digit_tweets:
    emoji_pattern = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "]+", flags=re.UNICODE)
    tweet = emoji_pattern.sub(r'', tweet)
    no_emoji_tweets.append(tweet)

no_emoji_tweets

['Volkswagen Ordered To Recall K Vehicles Over Its Own Malicious Programming',
 'What The Actual Fuck',
 'latimesphotos Volkswagen Have we just given up on English',
 'Seriously though what the fuck Volkswagen',
 'Volkswagen  Audi accused of using software to cheat US diesel emissions tests',
 'Jorge R Ruiz Volkswagen  Audi accused of using software to cheat US diesel emissions tests In a',
 'vwbeetle Volkswagen  Beetle  Classic  volkswagon convertible   BidEnd Date Friday Sep',
 'BLG VW cheated on emissions tests The EPA says Volkswagen installed software to cheat on emissions tests US',
 'VW cheated on emissions tests The EPA says Volkswagen installed software to cheat on emissions tests USA TODAYs Chris Woodyard rep',
 'VW cheated on emissions tests The EPA says Volkswagen installed software to cheat on emissions tests USA TOD',
 'EPA Volkswagen used defeat device to illegally skirt airpollution controls  Washington Post',
 'VW cheated on emissions tests The EPA says Volkswagen inst

In [11]:
#convert to lowercase
lower_tweets = []
for tweet in no_emoji_tweets : 
    tweet = tweet.lower()
    lower_tweets.append(tweet)
lower_tweets

['volkswagen ordered to recall k vehicles over its own malicious programming',
 'what the actual fuck',
 'latimesphotos volkswagen have we just given up on english',
 'seriously though what the fuck volkswagen',
 'volkswagen  audi accused of using software to cheat us diesel emissions tests',
 'jorge r ruiz volkswagen  audi accused of using software to cheat us diesel emissions tests in a',
 'vwbeetle volkswagen  beetle  classic  volkswagon convertible   bidend date friday sep',
 'blg vw cheated on emissions tests the epa says volkswagen installed software to cheat on emissions tests us',
 'vw cheated on emissions tests the epa says volkswagen installed software to cheat on emissions tests usa todays chris woodyard rep',
 'vw cheated on emissions tests the epa says volkswagen installed software to cheat on emissions tests usa tod',
 'epa volkswagen used defeat device to illegally skirt airpollution controls  washington post',
 'vw cheated on emissions tests the epa says volkswagen inst

In [12]:
#Split attached words (segmentation) :segmenter using the word statistics from Twitter

seg_tw = Segmenter(corpus="twitter")

no_attach_tweets = []
for tweet in lower_tweets:
    cleaned_tweet = seg_tw.segment(tweet)
    no_attach_tweets.append(cleaned_tweet)
no_attach_tweets

Reading twitter - 1grams ...
Reading twitter - 2grams ...


['volkswagen  ordered to  recall  k vehicles  over  its own malicious  programming',
 'what  the actual  fuck',
 'latimes photos  volkswagen have we  just  given up on  english',
 'seriously   though  what the fuck  volkswagen',
 'volkswagen   audi accused of  using  software to cheat u s diesel emissions  tests',
 'jorge r ruiz  volkswagen   audi  accused  of using  software  to cheat us diesel  emissions  tests in a',
 'vw beetle   volkswagen   beetle  classic   volkswagon  convertible   biden d date friday sep',
 'blg vw cheated on  emissions  tests the epa says  volkswagen  installed  software  to cheat on  emissions  tests us',
 'vw cheated on  emissions  tests the epa says  volkswagen  installed  software  to cheat on  emissions  tests usa  today s chris woodyard rep',
 'vw cheated on  emissions  tests the epa says  volkswagen  installed  software  to cheat on  emissions  tests usa tod',
 'epa  volkswagen  used defeat  device  to illegally skirt  air pollution  controls   washing

In [13]:
#Expand usual terms used in the field
expands_file ='expands.txt'
def get_list(file):
    with open(file, mode='r') as file:
        terms = csv.reader(file)
        return {rows[0]:rows[1] for rows in terms}
    
def expand_terms(words):
    expands = get_list(expands_file)
    for key in expands:
        words = words.replace(key, expands[key])
    return words

expand_tweets = []
for tweet in no_attach_tweets:
    tweet = expand_terms(tweet)
    expand_tweets.append(tweet)

expand_tweets

['volkswagen  ordered to  recall  k vehicles  over  its own malicious  programming',
 'what  the actual  fuck',
 'latimes photos  volkswagen have we  just  given up on  english',
 'seriously   though  what the fuck  volkswagen',
 'volkswagen   audi accused of  using  software to cheat u s diesel emissions  tests',
 ' Jorge-R-Ruiz  volkswagen   audi  accused  of using  software  to cheat us diesel  emissions  tests in a',
 ' volkswagen  beetle   volkswagen   beetle  classic   volkswagon  convertible   biden d date friday sep',
 'blg  volkswagen  cheated on  emissions  tests the Environmental-Protection-Agency says  volkswagen  installed  software  to cheat on  emissions  tests us',
 ' volkswagen  cheated on  emissions  tests the Environmental-Protection-Agency says  volkswagen  installed  software  to cheat on  emissions  tests united-states  today s chris woodyard rep',
 ' volkswagen  cheated on  emissions  tests the Environmental-Protection-Agency says  volkswagen  installed  software

In [14]:
#Standarizing words
standard_tweets = []
for tweet in expand_tweets :
    tweet = ''.join(''.join(s)[:2] for _, s in itertools.groupby(tweet))
    standard_tweets.append(tweet)
standard_tweets

['volkswagen  ordered to  recall  k vehicles  over  its own malicious  programming',
 'what  the actual  fuck',
 'latimes photos  volkswagen have we  just  given up on  english',
 'seriously  though  what the fuck  volkswagen',
 'volkswagen  audi accused of  using  software to cheat u s diesel emissions  tests',
 ' Jorge-R-Ruiz  volkswagen  audi  accused  of using  software  to cheat us diesel  emissions  tests in a',
 ' volkswagen  beetle  volkswagen  beetle  classic  volkswagon  convertible  biden d date friday sep',
 'blg  volkswagen  cheated on  emissions  tests the Environmental-Protection-Agency says  volkswagen  installed  software  to cheat on  emissions  tests us',
 ' volkswagen  cheated on  emissions  tests the Environmental-Protection-Agency says  volkswagen  installed  software  to cheat on  emissions  tests united-states  today s chris woodyard rep',
 ' volkswagen  cheated on  emissions  tests the Environmental-Protection-Agency says  volkswagen  installed  software  to ch

In [15]:
#Spelling correction
#Removal short words (length < 3)
correct_tweets = []
for tweet in standard_tweets : 
    correct_list = []
    word_list = tweet.split(' ')

    for word in word_list :
        if len(word)>2:
            correct_word = sp.correct(word)
            correct_list.append(correct_word)

    tweet = ' '.join(correct_list)
    correct_tweets.append(tweet)

correct_tweets

['volkswagen ordered recall vehicles over its own malicious programming',
 'what the actual fuck',
 'latimes photos volkswagen have just given english',
 'seriously though what the fuck volkswagen',
 'volkswagen audi accused using software cheat diesel emissions tests',
 'Jorge-R-Ruiz volkswagen audi accused using software cheat diesel emissions tests',
 'volkswagen beetle volkswagen beetle classic volkswagon convertible biden date friday sep',
 'blg volkswagen cheated emissions tests the Environmental-Protection-Agency says volkswagen installed software cheat emissions tests',
 'volkswagen cheated emissions tests the Environmental-Protection-Agency says volkswagen installed software cheat emissions tests unitedstates today chris woodyard rep',
 'volkswagen cheated emissions tests the Environmental-Protection-Agency says volkswagen installed software cheat emissions tests unitedstates tod',
 'epa volkswagen used defeat device illegally skirt air pollution controls washington post',
 'v

In [16]:
#Slangs lookup

In [17]:
#Removal of Stop-words
add_stopwords = ['http','https', 'www','blg','sep', 'iii', 'e.g.', 'i.e.', 'al.', 'com' , 'v.s.','bit','what','volkswagen','volkswagon','have','has','-PRON-','what','html','here','not']
all_stopwords = stopwords.words('english')+add_stopwords

non_stop_tweets = []
for tweet in correct_tweets : 
    word_list = tweet.split(' ')
    filtered_words = [word for word in word_list if word not in all_stopwords]
    tweet = ' '.join(filtered_words)
    non_stop_tweets.append(tweet)

non_stop_tweets

['ordered recall vehicles malicious programming',
 'actual fuck',
 'latimes photos given english',
 'seriously though fuck',
 'audi accused using software cheat diesel emissions tests',
 'Jorge-R-Ruiz audi accused using software cheat diesel emissions tests',
 'beetle beetle classic convertible biden date friday',
 'cheated emissions tests Environmental-Protection-Agency says installed software cheat emissions tests',
 'cheated emissions tests Environmental-Protection-Agency says installed software cheat emissions tests unitedstates today chris woodyard rep',
 'cheated emissions tests Environmental-Protection-Agency says installed software cheat emissions tests unitedstates tod',
 'epa used defeat device illegally skirt air pollution controls washington post',
 'cheated emissions tests Environmental-Protection-Agency says installed software cheat emissions tests unitedstates tod',
 'cheated emissions tests Environmental-Protection-Agency says installed software cheat emissions tests un

In [18]:
#Tokenization
def wsp_tokenizer(text):
    return text.split(" ")

wsp_tokens_tweets = []
for tweet in non_stop_tweets : 
    tweet = wsp_tokenizer(tweet)
    wsp_tokens_tweets.append(tweet)

wsp_tokens_tweets

[['ordered', 'recall', 'vehicles', 'malicious', 'programming'],
 ['actual', 'fuck'],
 ['latimes', 'photos', 'given', 'english'],
 ['seriously', 'though', 'fuck'],
 ['audi',
  'accused',
  'using',
  'software',
  'cheat',
  'diesel',
  'emissions',
  'tests'],
 ['Jorge-R-Ruiz',
  'audi',
  'accused',
  'using',
  'software',
  'cheat',
  'diesel',
  'emissions',
  'tests'],
 ['beetle', 'beetle', 'classic', 'convertible', 'biden', 'date', 'friday'],
 ['cheated',
  'emissions',
  'tests',
  'Environmental-Protection-Agency',
  'says',
  'installed',
  'software',
  'cheat',
  'emissions',
  'tests'],
 ['cheated',
  'emissions',
  'tests',
  'Environmental-Protection-Agency',
  'says',
  'installed',
  'software',
  'cheat',
  'emissions',
  'tests',
  'unitedstates',
  'today',
  'chris',
  'woodyard',
  'rep'],
 ['cheated',
  'emissions',
  'tests',
  'Environmental-Protection-Agency',
  'says',
  'installed',
  'software',
  'cheat',
  'emissions',
  'tests',
  'unitedstates',
  'tod']

In [19]:
#Bigram Detection
bigram = gensim.models.Phrases(wsp_tokens_tweets)
texts = [bigram[line] for line in wsp_tokens_tweets]
texts

/Users/faten/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:431: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


[['ordered', 'recall', 'vehicles', 'malicious', 'programming'],
 ['actual', 'fuck'],
 ['latimes', 'photos', 'given', 'english'],
 ['seriously', 'though', 'fuck'],
 ['audi',
  'accused',
  'using',
  'software',
  'cheat',
  'diesel',
  'emissions',
  'tests'],
 ['Jorge-R-Ruiz',
  'audi',
  'accused',
  'using',
  'software',
  'cheat',
  'diesel',
  'emissions',
  'tests'],
 ['beetle', 'beetle', 'classic', 'convertible', 'biden', 'date', 'friday'],
 ['cheated',
  'emissions',
  'tests',
  'Environmental-Protection-Agency',
  'says',
  'installed',
  'software',
  'cheat',
  'emissions',
  'tests'],
 ['cheated',
  'emissions',
  'tests',
  'Environmental-Protection-Agency',
  'says',
  'installed',
  'software',
  'cheat',
  'emissions',
  'tests',
  'unitedstates',
  'today',
  'chris',
  'woodyard',
  'rep'],
 ['cheated',
  'emissions',
  'tests',
  'Environmental-Protection-Agency',
  'says',
  'installed',
  'software',
  'cheat',
  'emissions',
  'tests',
  'unitedstates',
  'tod']